In [ ]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from scipy.stats import norm
from sklearn.linear_model import LassoCV, LinearRegression
import patsy
import warnings
import statsmodels.api as sm
warnings.simplefilter('ignore')
np.random.seed(1234)

# Application: Heterogeneous Effect of Sex on Wage Using Double Lasso

We use US census data from the year 2015 to analyse the effect of sex and interaction effects of other variables with sex on wage jointly. The dependent variable is the logarithm of the wage, the target variable is *female* (in combination with other variables). All other variables denote some other socio-economic characteristics, e.g. marital status, education, and experience.  For a detailed description of the variables we refer to the help page.



This analysis allows a closer look how discrimination according to sex is related to other socio-economic variables.

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
data = pd.read_csv(file)

In [ ]:
data.describe()

Define outcome and regressors

In [ ]:
y = np.log(data['wage']).values
Z = data.drop(['wage', 'lwage'], axis=1)
Z.columns

## Feature Engineering

Construct all our control variables

In [ ]:
# Ultra flexible controls of all pair-wise interactions (around 1k variables); un-comment to run this
Zcontrols = patsy.dmatrix('0 + (shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we+exp1+exp2+exp3+exp4)**2',
                           Z, return_type='dataframe')

Zcontrols = Zcontrols - Zcontrols.mean(axis=0)

Construct all the variables that we will use to model heterogeneity of effect in a linear manner

In [ ]:
Zhet = patsy.dmatrix('0 + (shs+hsg+scl+clg+mw+so+we+exp1+exp2+exp3+exp4)',
                      Z, return_type='dataframe')
Zhet = Zhet - Zhet.mean(axis=0)

Construct all interaction variables between sex and heterogeneity variables

In [ ]:
Zhet['sex'] = Z['sex']
Zinteractions = patsy.dmatrix('0 + sex + sex * (shs+hsg+scl+clg+mw+so+we+exp1+exp2+exp3+exp4)',
                               Zhet, return_type='dataframe')
interaction_cols = [c for c in Zinteractions.columns if c.startswith('sex')]

Put all the variables together

In [ ]:
X = pd.concat([Zinteractions, Zcontrols], axis=1)
X.shape

## Double Lasso for All Interactive Effects

Choose our penalized lasso regression model either cross validated lasso (see end of notebook for lasso based on the theoretically driven penalty)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=123)
lasso_model = lambda: make_pipeline(StandardScaler(), LassoCV(cv=cv))

For each target predictive effect estimate it via the partialling out process and calculate the quantities needed for the covariance calculation, which is the residual outcome, the residual target variable and the final stage residual epsilon.

In [ ]:
alpha = {}
res_y, res_D, epsilon = {}, {}, {}
for c in interaction_cols:
    print(f"Double Lasso for target variable {c}")
    D = X[c].values
    W = X.drop([c], axis=1)
    res_y[c] = y - lasso_model().fit(W, y).predict(W)
    res_D[c] = D - lasso_model().fit(W, D).predict(W)
    final = LinearRegression(fit_intercept=False).fit(res_D[c].reshape(-1, 1), res_y[c])
    epsilon[c] = res_y[c] - final.predict(res_D[c].reshape(-1, 1))
    alpha[c] = [final.coef_[0]]

## Co-Variance Calculation and Confidence Intervals

Calculate the covariance matrix of the estimated parameters

In [ ]:
V = np.zeros((len(interaction_cols), len(interaction_cols)))
for it, c in enumerate(interaction_cols):
    Jc = np.mean(res_D[c]**2)
    for itp, cp in enumerate(interaction_cols):
        Jcp = np.mean(res_D[cp]**2)
        Sigma = np.mean(res_D[c] * epsilon[c] * epsilon[cp] * res_D[cp])
        V[it, itp] = Sigma / (Jc * Jcp)

Calculate standard errors for each parameter

In [ ]:
n = X.shape[0]
for it, c in enumerate(interaction_cols):
    alpha[c] += [np.sqrt(V[it, it] / n)]

Put them all in a dataframe

In [ ]:
df = pd.DataFrame.from_dict(alpha, orient='index', columns=['point', 'stderr'])

Calculate and pointwise p-value

In [ ]:
summary = pd.DataFrame()
summary['Estimate'] = df['point']
summary['Std. Error'] = df['stderr']
summary['p-value'] = norm.sf(np.abs(df['point'] / df['stderr']), loc=0, scale=1) * 2
summary['ci_lower'] = df['point'] - 1.96 * df['stderr']
summary['ci_upper'] = df['point'] + 1.96 * df['stderr']

In [ ]:
summary

# Joint Confidence Intervals

In [ ]:
Drootinv = np.diagflat(1/np.sqrt(np.diag(V)))

In [ ]:
scaledCov = Drootinv @ V @ Drootinv

In [ ]:
np.random.seed(123)
U = np.random.multivariate_normal(np.zeros(scaledCov.shape[0]), scaledCov, size=10000)
z = np.max(np.abs(U), axis=1)
c = np.percentile(z, 95)
c

In [ ]:
summary = pd.DataFrame()
summary['Estimate'] = df['point']
summary['CI lower'] = df['point'] - c * df['stderr']
summary['CI upper'] = df['point'] + c * df['stderr']

In [ ]:
summary

We can also set the penalized lasso model to be estimated based on the theoretically motivated penalty level using the hdmpy package. To install it run
`pip install multiprocess`
`git clone https://github.com/maxhuppertz/hdmpy.git`

You can run the cells below and then repeat the whole analysis above using the newly defined `lasso_model` variable.

In [ ]:
import sys
sys.path.insert(1, "./hdmpy")

In [ ]:
# We wrap the package so that it has the familiar sklearn API
import hdmpy
from sklearn.base import BaseEstimator, clone

class RLasso(BaseEstimator):
    
    def __init__(self, *, post=True):
        self.post = post
    
    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])

lasso_model = lambda: RLasso(post=False)